<a href="https://colab.research.google.com/github/SURE18/Practical-Machine-Learning-with-Tensorflow/blob/master/Copy_of_Assignment_4_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Setup**

Install and import all the necessary libraries for the assignment.

In [1]:
!pip install tensorflow==2.0.0-rc0

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
import tensorflow as tf
import pandas as pd

tf.random.set_seed(1)

### **Importing the dataset**

In [0]:
boston_dataset = load_boston()

data_X = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
data_Y = pd.DataFrame(boston_dataset.target, columns=["target"])
data = pd.concat([data_X, data_Y], axis=1)

In [3]:
train, test = train_test_split(data, test_size=0.2, random_state=1)
train, val = train_test_split(train, test_size=0.2, random_state=1)
print(len(train), "train examples")
print(len(val), "validation examples")
print(len(test), "test examples")

323 train examples
81 validation examples
102 test examples


In [4]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
CRIM,323.0,3.882044,9.680410,0.00632,0.082865,0.25387,3.944055,88.9762
ZN,323.0,12.233746,24.323780,0.00000,0.000000,0.00000,19.000000,100.0000
INDUS,323.0,11.121796,6.876617,0.46000,5.190000,8.56000,18.100000,27.7400
CHAS,323.0,0.080495,0.272481,0.00000,0.000000,0.00000,0.000000,1.0000
NOX,323.0,0.553683,0.118625,0.38500,0.447500,0.53200,0.624000,0.8710
RM,323.0,6.276935,0.691051,3.56100,5.886500,6.19300,6.666500,8.7800
AGE,323.0,68.279876,28.452257,2.90000,43.550000,77.70000,93.900000,100.0000
DIS,323.0,3.864749,2.154972,1.12960,2.120350,3.36030,5.237900,12.1265
RAD,323.0,9.721362,8.801984,1.00000,4.000000,5.00000,24.000000,24.0000
TAX,323.0,406.588235,170.365557,187.00000,279.000000,329.00000,666.000000,711.0000


Converting the Pandas DataFrames into Tensorflow Datasets

In [0]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [6]:
batch_size = 32
train_ds = df_to_dataset(train, True, batch_size)
val_ds = df_to_dataset(val, False, batch_size)
test_ds = df_to_dataset(test, False, batch_size)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [59]:
data.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'target'],
      dtype='object')

### Defining Feature Columns

In [0]:
# define feature_columns as a list of features using functions from tf.feature_column

feature_columns = []
for key in feature_batch.keys():
    feature_columns.append(tf.feature_column.numeric_column(key))


In [50]:
feature_columns

[NumericColumn(key='CRIM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='ZN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='INDUS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='CHAS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NOX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='RM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='AGE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='DIS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='RAD', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='TAX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PTRATIO'

### Building the model

In [0]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

Model should contain following layers:

```
feature_layer

Dense(1, activation=None)
```

Use 'Adam' optimizer

Use 'mse' as your loss and metric

In [0]:
# Build and compile your model in this cell.
model = tf.keras.Sequential([
                             feature_layer,
                             tf.keras.layers.Dense(1,activation=None)])
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mean_squared_error'])

# model.compile(optimizer='adam',loss='mean_squared_error',metrics='mean_squared_error')

In [58]:
model.fit(train_ds, validation_data=val_ds, epochs=200)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/200
11/11 [==============================] - 1s 68ms/step - loss: 26289.7013 - mean_squared_error: 24809.9785 - val_loss: 0.0000e+00 - val_mean_squared_error: 0.0000e+00
Epoch 2/200
11/11 [==============================] - 0s 6ms/step - loss: 21972.5175 - mean_squared_error: 21943.3418 - val_loss: 17897.3880 - val_mean_squared_error: 18885.4219
Epoch 3/200
11/11 [==============================] - 0s 5ms/step - loss: 19495.4565 - mean_squared_error: 19372.2832 - val_loss: 15756.3008 - val_mean_sq

In [60]:
loss, mse = model.evaluate(train_ds)
print("Mean Squared Error - Test Data", mse)

11/11 [==============================] - 0s 18ms/step - loss: 220.7047 - mean_squared_error: 151.8126
Mean Squared Error - Test Data 151.81259


In [56]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features_2 (DenseFeatu multiple                  0         
_________________________________________________________________
dense_10 (Dense)             multiple                  14        
Total params: 14
Trainable params: 14
Non-trainable params: 0
_________________________________________________________________


In [62]:
import math
math.sqrt(mse)

12.3212252455969